### Building the baseline query engine

In [5]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.llms.openai import OpenAI
from dotenv import load_dotenv, find_dotenv



# Load environment variables when running locally
# Uncomment the next line if you have a .env file with your OpenAI API key
_ = load_dotenv(find_dotenv())


documents = SimpleDirectoryReader(input_files=["../data/Pmg_lds.md", "../data/handbook.md"]).load_data()
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()



In [6]:
import pandas as pd

df = pd.read_csv("../processed_data/processed/qa_pairs.csv")
df.tail()

,user_input,reference_contexts,reference
196,How do the teachings in 3 Nephi 11 and 3 Nephi...,['<1-hop>\n\nIdeas for Study and Application (...,3 Nephi 11 is significant in the context of th...
197,How does Ezra Taft Benson's emphasis on work a...,"['<1-hop>\n\nWork, work, work—there is no sati...",Ezra Taft Benson emphasized the importance of ...
198,How can the teachings in 1 Nephi 13:39 and 1 N...,['<1-hop>\n\nScripture Study (Page 22) How sho...,The teachings in 1 Nephi 13:39 and 1 Nephi 1 c...
199,How do D&C 84:85 and D&C 68:25 guide missionar...,['<1-hop>\n\nThe Power and Authority of Your C...,D&C 84:85 guides missionaries by emphasizing t...
200,How does D&C 42:12 emphasize the importance of...,['<1-hop>\n\nUse the Scriptures (Page 194) The...,D&C 42:12 emphasizes the importance of using s...


In [39]:
df["user_input"][1]

'How can missionaries help others come unto Christ according to the missionary lessons?'

In [40]:
response = query_engine.query(df["user_input"][1])
print(response)

Missionaries can help others come unto Christ by inviting them to exercise faith unto repentance, be baptized by water and the Spirit, and endure faithfully to the end. They can extend commitment invitations, follow up on commitments, and teach the principles of repentance and change. By showing love, faith in God's promises, and helping individuals repent and keep their commitments, missionaries can guide others towards coming unto Christ.


## Evaluating the Query Engine

In [41]:
from ragas.metrics import (
    Faithfulness,
    AnswerRelevancy,
    ContextPrecision,
    ContextRecall,
)

# init metrics with evaluator LLM
from ragas.llms import LlamaIndexLLMWrapper

evaluator_llm = LlamaIndexLLMWrapper(OpenAI(model="gpt-4o"))
metrics = [
    Faithfulness(llm=evaluator_llm),
    AnswerRelevancy(llm=evaluator_llm),
    ContextPrecision(llm=evaluator_llm),
    ContextRecall(llm=evaluator_llm),
]

In [52]:
from ragas.dataset_schema import EvaluationDataset

df["reference_contexts"] = df["reference_contexts"].apply(lambda s: [s] if pd.notna(s) else [])

# 4) Now create the EvaluationDataset
dataset: EvaluationDataset = EvaluationDataset.from_pandas(df)

In [53]:
dataset

EvaluationDataset(features=['user_input', 'reference_contexts', 'reference'], len=201)

In [ ]:
from ragas.integrations.llama_index import evaluate

result = evaluate(
    query_engine=query_engine,
    metrics=metrics,
    dataset=dataset,
)

In [55]:
result.to_pandas()

,user_input,retrieved_contexts,reference_contexts,response,reference,faithfulness,answer_relevancy,context_precision,context_recall
0,Wht is the significance of Moses 1:39 in missi...,[. . mine elders to be moving swiftly . . . wh...,[['## Preach My Gospel (D&C 50:14) (Page 5) | ...,Moses 1:39 emphasizes the importance of God's ...,Moses 1:39 is significant in missionary work a...,0.571429,0.909507,0.0,0.000000
1,How can missionaries help others come unto Chr...,"[Next, answer each question as though you were...",[['The Missionary Lessons (Page 9) You have th...,Missionaries can help others come unto Christ ...,Missionaries can help others come unto Christ ...,1.000000,0.980402,1.0,0.500000
2,Wht is the role of 'Jesus the Christ' in perso...,[- Identify an attribute you wish to study. Fo...,"[['Personal Study, Companion Study, District M...",The role of 'Jesus the Christ' in personal stu...,"In personal study, 'Jesus the Christ' is part ...",0.857143,0.993475,0.0,0.000000
3,What districts do in mission work?,[### District Meetings and Zone Conferences (P...,[['### Zone Conferences (Page 11) Zone confere...,Districts in mission work involve reading diff...,Many missionaries serve in branches and distri...,1.000000,0.958212,0.0,0.333333
4,How can 'Preach My Gospel' be utilized by ward...,[## Use by Ward Missionaries and Leaders (Page...,[['Use by Ward Missionaries and Leaders (Page ...,'Preach My Gospel' can be utilized by ward mis...,'Preach My Gospel' is primarily for the full-t...,1.000000,0.970110,1.0,1.000000
...,...,...,...,...,...,...,...,...,...
196,How do the teachings in 3 Nephi 11 and 3 Nephi...,[Those who have repented of their sins and rec...,[['<1-hop>\n\nIdeas for Study and Application ...,The teachings in 3 Nephi 11 and 3 Nephi 28:10 ...,3 Nephi 11 is significant in the context of th...,0.000000,0.993708,1.0,1.000000
197,How does Ezra Taft Benson's emphasis on work a...,[“In a world ever more dangerous . . . the Boo...,"[['<1-hop>\n\nWork, work, work—there is no sat...",Ezra Taft Benson's emphasis on work highlights...,Ezra Taft Benson emphasized the importance of ...,0.900000,0.941070,1.0,0.400000
198,How can the teachings in 1 Nephi 13:39 and 1 N...,[## The Message of the Restoration: The Founda...,[['<1-hop>\n\nScripture Study (Page 22) How sh...,1 Nephi 13:39 can be used to emphasize the imp...,The teachings in 1 Nephi 13:39 and 1 Nephi 1 c...,0.333333,0.952375,1.0,0.800000
199,How do D&C 84:85 and D&C 68:25 guide missionar...,[#### Scripture Study (Page 33)\nWhat do the s...,[['<1-hop>\n\nThe Power and Authority of Your ...,D&C 84:85 and D&C 68:25 guide missionaries in ...,D&C 84:85 guides missionaries by emphasizing t...,0.000000,0.999999,0.0,0.000000


In [56]:
print(result)

{'faithfulness': 0.6444, 'answer_relevancy': 0.9666, 'context_precision': 0.7114, 'context_recall': 0.6597}


### Evaluating with trulens

In [2]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
# Load environment variables when running locally
_ = load_dotenv(find_dotenv())

In [3]:
# uncomment if you need to activate the query engine
# from llama_index.core import VectorStoreIndex, SimpleDirectoryReader


documents = SimpleDirectoryReader(
    input_files=["../data/Pmg_lds.md", "../data/handbook.md"]
).load_data()

In [4]:
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()


In [5]:
response = query_engine.query("What is the purpose of the Preach my gospel?")
print(response)

The purpose of Preach My Gospel is to help missionaries be better-prepared, more spiritually mature, and more persuasive teachers. It is intended to assist missionaries in fulfilling their purpose by providing guidance on how to invite others to come unto Christ through faith in Jesus Christ and His Atonement, repentance, baptism, receiving the gift of the Holy Ghost, and enduring to the end.


In [57]:
response = query_engine.retrieve("What is the purpose of the Preach my gospel?")
print(response[0])

Node ID: 3731c076-9623-4f2c-a2fc-e0ff07bd8b9d
Text: ## Use by Ward Missionaries and Leaders (Page 13)  Preach My
Gospel is for the full-time missionaries of the Church. However, the
principles and doctrines taught herein are also applicable to ward
missionaries and leaders as they seek to build the Lord’s kingdom.
Frequent study of this manual will enable them to fulfill their
responsibilities as...
Score:  0.901



In [7]:
from trulens_eval import Tru

tru = Tru()
tru.reset_database()

/home/promise/projects/CSE499/Agentic_rag/.venv/lib/python3.12/site-packages/munch/__init__.py:24: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
/tmp/ipykernel_228176/1245245163.py:1: DeprecationWarning: The `trulens_eval` module is deprecated. See https://www.trulens.org/component_guides/other/trulens_eval_migration/ for instructions on migrating to `trulens.*` modules.
  from trulens_eval import Tru


🦑 Initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `TruSession` to prevent this.


Updating app_name and app_version in apps table: 0it [00:00, ?it/s]
Updating app_id in records table: 0it [00:00, ?it/s]
Updating app_json in apps table: 0it [00:00, ?it/s]


In [8]:
from trulens.core import TruSession

session = TruSession()
session.reset_database()

Updating app_name and app_version in apps table: 0it [00:00, ?it/s]
Updating app_id in records table: 0it [00:00, ?it/s]
Updating app_json in apps table: 0it [00:00, ?it/s]


In [9]:
# import os
# os.environ["TRULENS_OTEL_ENABLED"] = "true"

# After running this cell, restart your Jupyter kernel to ensure OTel is enabled.
# Then re-import all Trulens modules and re-run your code.

import numpy as np
from trulens.core import Feedback
from trulens.providers.openai import OpenAI
from trulens_eval import TruLlama

provider = OpenAI(model_engine="gpt-4.1-nano")

# Get context selector
context = TruLlama.select_context(query_engine)

# Define a groundedness feedback function
groundedness = (
    Feedback(
        provider.groundedness_measure_with_cot_reasons, name="Groundedness"
    )
    .on(context.collect())  # Use node.text from context
    .on_output()
)

# Question/answer relevance between overall question and answer.
answer_relevance = (
    Feedback(provider.relevance_with_cot_reasons, name="Answer Relevance")
    .on_input()
    .on_output()
)

# Context relevance between question and each context chunk.
context_relevance = (
    Feedback(
        provider.context_relevance_with_cot_reasons, name="Context Relevance"
    )
    .on_input()
    .on(context)  # Use node.text from context
    .aggregate(np.mean)
)

✅ In Groundedness, input source will be set to __record__.calls[-1].rets.source_nodes[:].node.text.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input context will be set to __record__.calls[-1].rets.source_nodes[:].node.text .


In [13]:
tru_recorder = TruLlama(
    query_engine,
    app_name="LlamaIndex_App",
    app_version="Baseline Evaluation",
    feedbacks=[groundedness, answer_relevance, context_relevance],
)

instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for base <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'>
instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for base <class 'llama_index.core.base.embeddings.base.BaseEmbedding'>
instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for base <class 'llama_index.core.schema.TransformComponent'>
instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for base <class 'llama_index.core.schema.BaseComponent'>
instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for base <class 'pydantic.main.BaseModel'>
instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for base <class 'llama_index_instrumentation.DispatcherSpanMixin'>
instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for base <class 'abc.ABC'>
instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for

In [14]:
eval_questions = df['user_input'].tolist()
for question in eval_questions[:2]:
    print(question)
print(type(eval_questions))

Wht is the significance of Moses 1:39 in missionary work?
How can missionaries help others come unto Christ according to the missionary lessons?
<class 'list'>


In [15]:
for question in eval_questions[:10]:
    with tru_recorder as recording:
        response = query_engine.query(question)

In [16]:
records, feedback = tru.get_records_and_feedback(app_ids=[])
records.head()

,app_name,app_version,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,latency,total_tokens,total_cost,cost_currency
0,LlamaIndex_App,Baseline Evaluation,app_hash_767c9103bd0233e2617644e82bbaccd7,"{'tru_class_info': {'name': 'TruLlama', 'modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_bfcef6c5b1baa517c1d3c381be74a13e,How does Moroni 10:3–5 play a role in helping ...,Moroni 10:3–5 plays a role in helping individu...,-,{'record_id': 'record_hash_bfcef6c5b1baa517c1d...,"{""n_requests"": 2, ""n_successful_requests"": 2, ...","{""start_time"": ""2025-07-16T22:51:14.569487"", ""...",2025-07-16T22:51:16.804716,2.233601,2088,0.003193,USD
1,LlamaIndex_App,Baseline Evaluation,app_hash_767c9103bd0233e2617644e82bbaccd7,"{'tru_class_info': {'name': 'TruLlama', 'modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_4570a3df8d75abfe376ffb6a8a5bce09,What 2 Nephi 31:10–21 say about gospel of Christ?,2 Nephi 31:10–21 discusses the importance of f...,-,{'record_id': 'record_hash_4570a3df8d75abfe376...,"{""n_requests"": 2, ""n_successful_requests"": 2, ...","{""start_time"": ""2025-07-16T22:51:12.658544"", ""...",2025-07-16T22:51:14.150586,1.490289,1797,0.002725,USD
2,LlamaIndex_App,Baseline Evaluation,app_hash_767c9103bd0233e2617644e82bbaccd7,"{'tru_class_info': {'name': 'TruLlama', 'modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_4145af19192f3a1bcf41069440fa5eb9,Wht does 2 Nephi say abot the gospel of Jesus ...,"2 Nephi discusses the gospel of Jesus Christ, ...",-,{'record_id': 'record_hash_4145af19192f3a1bcf4...,"{""n_requests"": 2, ""n_successful_requests"": 2, ...","{""start_time"": ""2025-07-16T22:51:10.026638"", ""...",2025-07-16T22:51:12.258367,2.230203,1929,0.002926,USD
3,LlamaIndex_App,Baseline Evaluation,app_hash_767c9103bd0233e2617644e82bbaccd7,"{'tru_class_info': {'name': 'TruLlama', 'modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_33c28eb689ed7ee27056d40b5ac22351,What D&C 79:1 say about missionary work?,D&C 79:1 emphasizes the importance of missiona...,-,{'record_id': 'record_hash_33c28eb689ed7ee2705...,"{""n_requests"": 2, ""n_successful_requests"": 2, ...","{""start_time"": ""2025-07-16T22:51:07.624379"", ""...",2025-07-16T22:51:09.586933,1.960022,2218,0.003340,USD
4,LlamaIndex_App,Baseline Evaluation,app_hash_767c9103bd0233e2617644e82bbaccd7,"{'tru_class_info': {'name': 'TruLlama', 'modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_687756489710e90d18d5584ee2ca6baa,How does the message in Romans 1:16 relate to ...,The message in Romans 1:16 relates to the teac...,-,{'record_id': 'record_hash_687756489710e90d18d...,"{""n_requests"": 2, ""n_successful_requests"": 2, ...","{""start_time"": ""2025-07-16T22:51:05.073621"", ""...",2025-07-16T22:51:07.248421,2.171883,1907,0.002895,USD


In [17]:
import pandas as pd

pd.set_option("display.max_colwidth", None)
records[["input", "output"] + feedback]

,input,output
0,How does Moroni 10:3–5 play a role in helping individuals gain a spiritual confirmation of the Restoration of the gospel?,"Moroni 10:3–5 plays a role in helping individuals gain a spiritual confirmation of the Restoration of the gospel by providing a promise that those who sincerely read and pray about the Book of Mormon will know of its truthfulness through the power of the Holy Ghost. This promise encourages individuals to engage in personal study and prayer, seeking divine confirmation of the Book of Mormon's authenticity and Joseph Smith's role as a prophet in the Restoration. By following the guidance in Moroni 10:3–5, individuals can receive a spiritual witness that strengthens their faith in the Restoration and the gospel of Jesus Christ."
1,What 2 Nephi 31:10–21 say about gospel of Christ?,"2 Nephi 31:10–21 discusses the importance of following Jesus Christ's example by being baptized, receiving the Holy Ghost, and enduring faithfully to the end. It emphasizes the need for repentance, baptism, and following the Savior's teachings to receive salvation through His Atonement."
2,Wht does 2 Nephi say abot the gospel of Jesus Christ?,"2 Nephi discusses the gospel of Jesus Christ, emphasizing the importance of faith in Christ, repentance, baptism, receiving the Holy Ghost, and enduring to the end. It highlights the principles of the gospel that individuals need to follow to be cleansed of their sins and receive the Savior's mercy at the day of judgment."
3,What D&C 79:1 say about missionary work?,D&C 79:1 emphasizes the importance of missionary work and the responsibility of every member to actively participate in sharing the gospel message.
4,How does the message in Romans 1:16 relate to the teaching of the gospel by the Spirit?,"The message in Romans 1:16 relates to the teaching of the gospel by the Spirit by emphasizing the power of the gospel to bring salvation to everyone who believes. This aligns with the teaching that the Spirit is essential in preaching the Gospel, as it is through the Spirit that individuals can come to faith and understanding of the message of salvation."
5,How can 'Preach My Gospel' be utilized by ward missionaries and leaders to support their responsibilities and foster unity with full-time missionaries?,"Ward missionaries and leaders can utilize 'Preach My Gospel' to fulfill their responsibilities as member missionaries by studying the manual frequently. This will enable them to apply the principles and doctrines taught in the manual to help build the Lord's kingdom. Additionally, studying 'Preach My Gospel' will help ward missionaries and leaders foster unity with full-time missionaries as they work together in their missionary efforts."
6,What districts do in mission work?,"Districts in mission work engage in activities such as discussing goals and plans to help individuals progress towards baptism and confirmation, sharing experiences to strengthen faith in the Savior, repentance, and receiving baptism, preparing for proselyting in new areas, conducting training sessions, and integrating investigator families into the ward. They also focus on setting goals, making plans, and emphasizing the importance of planning meaningful activities to effectively carry out missionary work."
7,Wht is the role of 'Jesus the Christ' in personal study?,"The role of 'Jesus the Christ' in personal study is to serve as the focal point of faith and belief. By studying His teachings, understanding His mission, and developing a relationship with Him through scripture study, prayer, and obedience, individuals can strengthen their faith in Him, receive personal inspiration, and ultimately grow closer to Him. Jesus the Christ is the central figure that individuals look to for guidance, forgiveness, and salvation, making Him the cornerstone of personal study and spiritual growth."
8,How can missionaries help others come unto Christ according to the missionary lessons?,"Missionaries can help others come u

In [3]:
import numpy as np
import nest_asyncio
nest_asyncio.apply()

# from trulens_eval import OpenAI as fOpenAI
from trulens.core import Feedback
from trulens_eval import TruLlama
from trulens_eval import FeedbackMode
from trulens.providers.openai import OpenAI

In [66]:

provider = OpenAI(model_engine="gpt-4o")
context = TruLlama.select_context(query_engine)

In [67]:
# set up feedback function for answer relevance - how relevant is the answer to the question

f_answer_relevance = (
    Feedback(
        provider.relevance_with_cot_reasons, name="Answer Relevance")
    .on_input()
    .on_output()
)

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .


In [68]:
# set up feedback function for context relevance - how relevant is the context to the question

context_selection = TruLlama.select_source_nodes().node.text

f_context_relevance = (
    Feedback(
        provider.relevance_with_cot_reasons, name="Context Relevance")
    .on_input()
    .on(context)
    .aggregate(np.mean)
)

✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.calls[-1].rets.source_nodes[:].node.text .


In [69]:
# set up feedback function for groundedness - how grounded is the answer to the context

# grounded = Groundedness(groundedness_provider=provider)

f_groundedness = (
    Feedback(
        provider.groundedness_measure_with_cot_reasons, name="Groundedness"
    )
    .on(context.collect())
    .on_output()
    # .aggregate(np.mean)
)

✅ In Groundedness, input source will be set to __record__.calls[-1].rets.source_nodes[:].node.text.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [34]:
from trulens.apps.llamaindex import TruLlama

tru_rag = TruLlama(
    query_engine,
    app_name="OTEL-RAG",
    app_version="4.1-mini",
    feedbacks=[groundedness, answer_relevance, context_relevance],
)

instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for base <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'>
instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for base <class 'llama_index.core.base.embeddings.base.BaseEmbedding'>
instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for base <class 'llama_index.core.schema.TransformComponent'>
instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for base <class 'llama_index.core.schema.BaseComponent'>
instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for base <class 'pydantic.main.BaseModel'>
instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for base <class 'llama_index_instrumentation.DispatcherSpanMixin'>
instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for base <class 'abc.ABC'>
instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for

In [27]:
tru_recorder = TruLlama(
    query_engine,
    app_id="baseline_eval_app_1",
    feedbacks=[
        f_answer_relevance,
        f_context_relevance,
        f_groundedness,
    ]
    # feedback_mode=FeedbackMode.ASYNC,  # Use async mode for better performance
)

instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for base <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'>
instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for base <class 'llama_index.core.base.embeddings.base.BaseEmbedding'>
instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for base <class 'llama_index.core.schema.TransformComponent'>
instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for base <class 'llama_index.core.schema.BaseComponent'>
instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for base <class 'pydantic.main.BaseModel'>
instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for base <class 'llama_index_instrumentation.DispatcherSpanMixin'>
instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for base <class 'abc.ABC'>
instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for

In [10]:
# eval_questions = []

eval_questions = df['user_input'].tolist()
for question in eval_questions[:2]:
    print(question)
print(type(eval_questions))

Wht is the significance of Moses 1:39 in missionary work?
How can missionaries help others come unto Christ according to the missionary lessons?
<class 'list'>


In [36]:
for question in eval_questions[:5]:
    with tru_rag as recording:
        query_engine.query(question)

In [37]:
records, feedback = tru.get_records_and_feedback(app_ids=[])
records.head()

,app_name,app_version,app_id,app_json,type,record_id,input,output,tags,record_json,...,Answer Relevance_calls,Context Relevance_calls,Groundedness_calls,Answer Relevance feedback cost in USD,Context Relevance feedback cost in USD,Groundedness feedback cost in USD,latency,total_tokens,total_cost,cost_currency
0,baseline_eval_app_1,base,app_hash_2c3eda9736b1795bf96f114682658693,"{'tru_class_info': {'name': 'TruLlama', 'modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_898fb4ef9d7f616f34be6107a23a8e0f,How can 'Preach My Gospel' be utilized by ward...,'Preach My Gospel' can be utilized by ward mis...,-,{'record_id': 'record_hash_898fb4ef9d7f616f34b...,...,[],[],[],0.0,0.0,0.0,2.370576,2105,0.003193,USD
1,baseline_eval_app_1,base,app_hash_2c3eda9736b1795bf96f114682658693,"{'tru_class_info': {'name': 'TruLlama', 'modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_464367ab23826d4d84972b650fcae762,What districts do in mission work?,Districts in mission work involve reading diff...,-,{'record_id': 'record_hash_464367ab23826d4d849...,...,[],[],[],0.0,0.0,0.0,2.361987,2135,0.003244,USD
2,baseline_eval_app_1,base,app_hash_2c3eda9736b1795bf96f114682658693,"{'tru_class_info': {'name': 'TruLlama', 'modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_9da9b570a3d516e0f2e7205b6e2567c4,Wht is the role of 'Jesus the Christ' in perso...,The role of 'Jesus the Christ' in personal stu...,-,{'record_id': 'record_hash_9da9b570a3d516e0f2e...,...,[],[],[],0.0,0.0,0.0,3.019258,1974,0.003026,USD
3,baseline_eval_app_1,base,app_hash_2c3eda9736b1795bf96f114682658693,"{'tru_class_info': {'name': 'TruLlama', 'modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_7fbd728e649edb8afeca03a8dbff6fa3,How can missionaries help others come unto Chr...,Missionaries can help others come unto Christ ...,-,{'record_id': 'record_hash_7fbd728e649edb8afec...,...,[],[],[],0.0,0.0,0.0,2.788594,2205,0.003354,USD
4,baseline_eval_app_1,base,app_hash_2c3eda9736b1795bf96f114682658693,"{'tru_class_info': {'name': 'TruLlama', 'modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_edf0c39e397c0f7b885257ee3e7cb7bd,Wht is the significance of Moses 1:39 in missi...,Moses 1:39 emphasizes the importance of God's ...,-,{'record_id': 'record_hash_edf0c39e397c0f7b885...,...,[],[],[],0.0,0.0,0.0,9.281551,2268,0.003442,USD


In [76]:
session.get_leaderboard()

,,latency,total_cost
app_name,app_version,,
baseline_eval_app_1,base,3.748015,0.003256


# old application... delete

In [33]:
import numpy as np
from trulens.apps.llamaindex import TruLlama
from trulens.core import Feedback
from trulens.providers.openai import OpenAI

# Initialize provider class
provider = OpenAI(model_engine="gpt-4o")

# select context to be used in feedback. the location of context is app specific.

context = TruLlama.select_context(query_engine)

# Define a groundedness feedback function
f_groundedness = (
    Feedback(
        provider.groundedness_measure_with_cot_reasons, name="Groundedness"
    )
    .on(context.collect())  # collect context chunks into a list
    .on_output()
)

# Question/answer relevance between overall question and answer.
f_answer_relevance = Feedback(
    provider.relevance_with_cot_reasons, name="Answer Relevance"
).on_input_output()
# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(
        provider.context_relevance_with_cot_reasons, name="Context Relevance"
    )
    .on_input()
    .on(context)
    .aggregate(np.mean)
)

✅ In Groundedness, input source will be set to __record__.calls[-1].rets.source_nodes[:].node.text.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input context will be set to __record__.calls[-1].rets.source_nodes[:].node.text .


In [34]:
tru_query_engine_recorder = TruLlama(
    query_engine,
    app_name="LlamaIndex_App",
    app_version="Direct Query Engine",
    feedbacks=[f_groundedness, f_answer_relevance, f_context_relevance],
)

instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for base <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'>
instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for base <class 'llama_index.core.base.embeddings.base.BaseEmbedding'>
instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for base <class 'llama_index.core.schema.TransformComponent'>
instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for base <class 'llama_index.core.schema.BaseComponent'>
instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for base <class 'pydantic.main.BaseModel'>
instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for base <class 'llama_index_instrumentation.DispatcherSpanMixin'>
instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for base <class 'abc.ABC'>
instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for

In [36]:
questions = df["user_input"].tolist() 
len(questions)

201

In [37]:
for question in questions:
    with tru_query_engine_recorder as recording:
        response = query_engine.query(question)


/home/promise/projects/CSE499/Agentic_rag/.venv/lib/python3.12/site-packages/trulens/feedback/llm_provider.py:1798: UserWarning: Failed to process and remove trivial statements. Proceeding with all statements.
  warnings.warn(


/home/promise/projects/CSE499/Agentic_rag/.venv/lib/python3.12/site-packages/trulens/feedback/llm_provider.py:1798: UserWarning: Failed to process and remove trivial statements. Proceeding with all statements.
  warnings.warn(
[nltk_data] Error loading punkt_tab: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>
[nltk_data] Error loading punkt_tab: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>[nltk_data] Error loading punkt_tab: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>

[nltk_data] Error loading punkt_tab: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>


In [38]:
records, feedback = tru.get_records_and_feedback(app_ids=[])

In [39]:
tru.get_leaderboard()

,,Groundedness,latency,total_cost
app_name,app_version,,,
LlamaIndex_App,Direct Query Engine,0.0,2.870236,0.003195


In [42]:
records.head()

,app_name,app_version,app_id,app_json,type,record_id,input,output,tags,record_json,...,Groundedness_calls,Groundedness feedback cost in USD,Answer Relevance_calls,Answer Relevance feedback cost in USD,Context Relevance_calls,Context Relevance feedback cost in USD,latency,total_tokens,total_cost,cost_currency
0,LlamaIndex_App,Direct Query Engine,app_hash_1b733ef7cda782608cf03d2d6ab60e01,"{'tru_class_info': {'name': 'TruLlama', 'modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_f8cb323533dc06563b8496cb15e83886,How does D&C 42:12 emphasize the importance of...,D&C 42:12 emphasizes the importance of using s...,-,{'record_id': 'record_hash_f8cb323533dc06563b8...,...,NaN,NaN,NaN,NaN,NaN,NaN,2.157995,2179,0.003330,USD
1,LlamaIndex_App,Direct Query Engine,app_hash_1b733ef7cda782608cf03d2d6ab60e01,"{'tru_class_info': {'name': 'TruLlama', 'modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_80cba94c0d9f42b790fd7f61e1e5d2db,How do D&C 84:85 and D&C 68:25 guide missionar...,D&C 84:85 and D&C 68:25 guide missionaries in ...,-,{'record_id': 'record_hash_80cba94c0d9f42b790f...,...,NaN,NaN,NaN,NaN,NaN,NaN,1.918074,1661,0.002545,USD
2,LlamaIndex_App,Direct Query Engine,app_hash_1b733ef7cda782608cf03d2d6ab60e01,"{'tru_class_info': {'name': 'TruLlama', 'modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_a562531bd09feac45ab6764b8b45b0fe,How can the teachings in 1 Nephi 13:39 and 1 N...,1 Nephi 13:39 can be used to emphasize the imp...,-,{'record_id': 'record_hash_a562531bd09feac45ab...,...,NaN,NaN,NaN,NaN,NaN,NaN,1.973820,2106,0.003218,USD
3,LlamaIndex_App,Direct Query Engine,app_hash_1b733ef7cda782608cf03d2d6ab60e01,"{'tru_class_info': {'name': 'TruLlama', 'modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_35bfef7806b6a7de842a9ff9fc336a65,How does Ezra Taft Benson's emphasis on work a...,Ezra Taft Benson's emphasis on work highlights...,-,{'record_id': 'record_hash_35bfef7806b6a7de842...,...,NaN,NaN,NaN,NaN,NaN,NaN,2.737495,2103,0.003222,USD
4,LlamaIndex_App,Direct Query Engine,app_hash_1b733ef7cda782608cf03d2d6ab60e01,"{'tru_class_info': {'name': 'TruLlama', 'modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_2e8faa61e255be9f672d9758cef87874,How do the teachings in 3 Nephi 11 and 3 Nephi...,The teachings in 3 Nephi 11 and 3 Nephi 28:10 ...,-,{'record_id': 'record_hash_2e8faa61e255be9f672...,...,NaN,NaN,NaN,NaN,NaN,NaN,2.844890,2204,0.003361,USD


In [65]:
import pandas as pd

pd.set_option("display.max_colwidth", None)
records[["input", "output"] + feedback]

,input,output,Groundedness
0,"How does D&C 42:12 emphasize the importance of using scriptures in teaching, and what role does D&C 42:14 play in ensuring the power of the Spirit in missionary work?","D&C 42:12 emphasizes the importance of using scriptures in teaching by instructing individuals to teach the principles of the gospel as found in the scriptures. This highlights the significance of grounding teachings in scriptural truths. On the other hand, D&C 42:14 plays a role in ensuring the power of the Spirit in missionary work by guiding individuals to rely on the Spirit for what they should teach and how they should teach it. This reliance on the Spirit is crucial for effective missionary work and helps ensure that teachings are inspired and impactful.",NaN
1,How do D&C 84:85 and D&C 68:25 guide missionaries in teaching the gospel and strengthening families?,"D&C 84:85 guides missionaries in teaching the gospel by emphasizing the importance of teaching the principles of the gospel with the Spirit, and not with contention. D&C 68:25 guides missionaries in strengthening families by encouraging them to teach their children to understand the doctrine of repentance, faith in Christ, baptism, and the gift of the Holy Ghost.",NaN
2,"How can the teachings in 1 Nephi 13:39 and 1 Nephi 1 be used to enhance missionary work and confirm the truthfulness of the message of the Restoration, as suggested in the Preach My Gospel manual?","1 Nephi 13:39 can be used to emphasize the importance of the Book of Mormon as a key tool in missionary work, highlighting its role in teaching the message of the Restoration. Additionally, 1 Nephi 1 can be utilized to showcase the experience of Nephi receiving revelation and guidance from God, which can inspire missionaries to seek similar divine guidance in their efforts. By incorporating these teachings into missionary work, individuals can strengthen their testimonies of the Restoration and effectively convey the message to others, as outlined in the Preach My Gospel manual.",NaN
3,How does Ezra Taft Benson's emphasis on work and the Book of Mormon's answers to the great questions of the soul guide missionaries in their efforts to teach and live the gospel?,"Ezra Taft Benson's emphasis on work highlights the importance of sharing the gospel with others and viewing every member as a missionary. By focusing on the Book of Mormon's answers to the great questions of the soul, missionaries are guided to help individuals find spiritual answers and develop faith in Jesus Christ. This approach encourages missionaries to use the Book of Mormon to address objections, strengthen testimonies, and guide individuals towards personal revelation. Ultimately, by combining Benson's emphasis on missionary work with the Book of Mormon's teachings, missionaries are equipped to effectively teach and live the gospel by helping individuals draw nearer to God and find answers to their spiritual inquiries.",NaN
4,How do the teachings in 3 Nephi 11 and 3 Nephi 28:10 relate to the understanding of the plan of salvation and the kingdoms of glory as described in the scriptures?,"The teachings in 3 Nephi 11 and 3 Nephi 28:10 relate to the understanding of the plan of salvation and the kingdoms of glory as described in the scriptures by emphasizing the importance of baptism and the different degrees of glory that individuals can attain based on their acceptance of the gospel and their actions in mortality. These teachings highlight the significance of following the example of Jesus Christ through baptism and receiving the ordinances of the gospel to progress towards exaltation in the highest kingdom, the celestial kingdom, or to receive placement in the terrestrial or telestial kingdoms based on individual choices and actions.",NaN
...,...,...,...
196,How can 'Preach My Gospel' be utilized by ward missionaries and leaders to support their responsibilities and foster unity with full-time missionaries?,Ward missionaries and leaders can 